In [21]:
import requests
import json
from folium import Map, TileLayer
from io import BytesIO
import rasterio
import urllib

%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [127]:
titiler_endpoint = "https://tt.resat.dev-vizzuality.com"  
#titiler_endpoint = "http://127.0.0.1:3000"
# Two sample COGs - one being a PSScene analytic image and the other being a PSOrthoTile visual image
# from the Google Cloud storage bucket
cog_url = "https://resilience-atlas.s3.amazonaws.com/cog/globcover_cog_test.tif"
cog_url = "https://oin-hotosm.s3.amazonaws.com/56f9b5a963ebf4bc00074e70/0/56f9c2d42b67227a79b4faec.tif"

In [128]:
def build_url(BASE_URL, endpoint, params):
    str_params = urllib.parse.urlencode(params, safe='/:"\'#', quote_via=urllib.parse.quote)
    return f"{BASE_URL}/{endpoint}?{str_params}"

def submit_request(BASE_URL, endpoint, url):
    r = requests.get(
        f"{BASE_URL}/{endpoint}",
        params = {
            "url": url,
        }
    ).json()
    return r

In [129]:
analytic_cog_validation = submit_request(titiler_endpoint, "info", cog_url)
print(json.dumps(analytic_cog_validation, indent=1))

{
 "bounds": [
  168.45251984367613,
  -17.557596404471102,
  168.459511241736,
  -17.54927007461763
 ],
 "minzoom": 15,
 "maxzoom": 22,
 "band_metadata": [
  [
   "b1",
   {}
  ],
  [
   "b2",
   {}
  ],
  [
   "b3",
   {}
  ]
 ],
 "band_descriptions": [
  [
   "b1",
   ""
  ],
  [
   "b2",
   ""
  ],
  [
   "b3",
   ""
  ]
 ],
 "dtype": "uint8",
 "nodata_type": "None",
 "colorinterp": [
  "red",
  "green",
  "blue"
 ],
 "overviews": [
  2,
  4,
  8,
  16,
  32,
  64
 ],
 "height": 25201,
 "count": 3,
 "driver": "GTiff",
 "width": 20179
}


In [130]:
tile_json_analytic = submit_request(titiler_endpoint, "tilejson.json", cog_url)
print(json.dumps(tile_json_analytic, indent=4))

{
    "tilejson": "2.2.0",
    "version": "1.0.0",
    "scheme": "xyz",
    "tiles": [
        "https://tt.resat.dev-vizzuality.com/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?url=https%3A%2F%2Foin-hotosm.s3.amazonaws.com%2F56f9b5a963ebf4bc00074e70%2F0%2F56f9c2d42b67227a79b4faec.tif"
    ],
    "minzoom": 15,
    "maxzoom": 22,
    "bounds": [
        168.45251984367613,
        -17.557596404471102,
        168.459511241736,
        -17.54927007461763
    ],
    "center": [
        168.45601554270607,
        -17.55343323954437,
        15
    ]
}


In [131]:
json.dumps({"11":"#AAF0F0","14":"#FFFF64","20":"#DCF064","30":"#CDCD66","40":"#006400","50":"#00A000","60":"#AAC800","70": "#003C00","90":"#286400","100":"#788200","110":"#8CA000","120":"#BE9600","130":"#966400","140":"#FFB432","150":"#FFEBAF","160":"#00785A","170":"#009678","180":"#00DC82","190":"#C31400","200":"#FFF5D7","210":"#0046C8","220":"#FFFFFF","230":"#000000"}, indent=0)

'{\n"11": "#AAF0F0",\n"14": "#FFFF64",\n"20": "#DCF064",\n"30": "#CDCD66",\n"40": "#006400",\n"50": "#00A000",\n"60": "#AAC800",\n"70": "#003C00",\n"90": "#286400",\n"100": "#788200",\n"110": "#8CA000",\n"120": "#BE9600",\n"130": "#966400",\n"140": "#FFB432",\n"150": "#FFEBAF",\n"160": "#00785A",\n"170": "#009678",\n"180": "#00DC82",\n"190": "#C31400",\n"200": "#FFF5D7",\n"210": "#0046C8",\n"220": "#FFFFFF",\n"230": "#000000"\n}'

In [132]:
# "colormap": json.dumps({"11":"#AAF0F0","14":"#FFFF64","20":"#DCF064","30":"#CDCD66","40":"#006400","50":"#00A000","60":"#AAC800","70": "#003C00","90":"#286400","100":"#788200","110":"#8CA000","120":"#BE9600","130":"#966400","140":"#FFB432","150":"#FFEBAF","160":"#00785A","170":"#009678","180":"#00DC82","190":"#C31400","200":"#FFF5D7","210":"#0046C8","220":"#FFFFFF","230":"#000000"}),
#    "colormap_type":"linear"

params = {
    "url": cog_url
}
tileset = build_url(titiler_endpoint, "tiles/{z}/{x}/{y}.png", params)
#tileset = tile_json_analytic.get("tiles", [])[0]
print(tileset)
(left, bottom, right, top) = tile_json_analytic["bounds"]
(lon,lat,zoom) = tile_json_analytic.get("center", [0, 0, 2])
center = [lat,lon]
m = Map(
    location=center,
    zoom_start=zoom,
    tiles="OpenStreetMap"
)
tile_layer = TileLayer(
    tiles= tileset,
    attr="Sample COG"
)

tile_layer.add_to(m)
m

https://tt.resat.dev-vizzuality.com/tiles/{z}/{x}/{y}.png?url=https://oin-hotosm.s3.amazonaws.com/56f9b5a963ebf4bc00074e70/0/56f9c2d42b67227a79b4faec.tif


In [97]:
!curl https://tt.resat.dev-vizzuality.com/tiles/5/18/13.png?url=https://resilience-atlas.s3.amazonaws.com/cog/globcover_cog_test.tif&bidx=1&colormap_name=viridis

In [ ]:
http://127.0.0.1:3000/tiles/{z}/{x}/{y}.png?url=https://oin-hotosm.s3.amazonaws.com/56f9b5a963ebf4bc00074e70/0/56f9c2d42b67227a79b4faec.tif